In [ ]:
%matplotlib ipympl
from ipymesh import PSLGEditor
from proteus import Domain

class PSLG_DomainEditor(PSLGEditor, Domain.PlanarStraightLineGraphDomain):
    def __init__(self, *args, **kwargs):
        super(PSLG_DomainEditor, self).__init__(*args, **kwargs)

In [ ]:
deditor = PSLG_DomainEditor(vertices=[[0.1, 0.1], 
                                      [0.1, 0.9], 
                                      [0.9120345052083333, 0.5249609375], 
                                      [0.618701171875, 0.5049609375], 
                                      [0.6220345052083334, 0.3216276041666667], 
                                      [0.913701171875, 0.33329427083333335], 
                                      [0.918701171875, 0.1], 
                                      [0.898701171875, 0.9082942708333334], 
                                      [0.23536786397298176, 0.6149609375], 
                                      [0.37036786397298177, 0.4882942708333333], 
                                      [0.4553678385416667, 0.5916276041666667], 
                                      [0.3203678639729818, 0.7132942708333333]], 
                            vertexFlags=[1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], 
                            segments=[[0, 1], [5, 6], [4, 5], [3, 4], [2, 3], 
                                      [2, 7], [1, 7], [8, 11], [8, 9], [9, 10], 
                                      [10, 11], [0, 6]], 
                            segmentFlags=[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1], 
                            regions=[(0.4, 0.75)], 
                            regionFlags=[1], 
                            regionTypes=[1, 2, 0],
                            holes=[[0.3487011973063151, 0.5881998697916667]],
                            boundaryTypes=[1,2,0],
                            regionConstraints=None)
deditor

In [ ]:
from proteus import TriangleTools
deditor.writePoly('mesh')
tmesh = TriangleTools.TriangleBaseMesh(baseFlags='VApq30Dena{0:f}'.format(0.01),
                                       nbase=1,
                                       verbose=10)
tmesh.readFromPolyFile(deditor.polyfile)
tmesh.writeToFile(deditor.polyfile)
mesh=tmesh.convertToProteusMesh(verbose=1)

In [ ]:
from matplotlib import pyplot as plt
plt.figure()
cmap = plt.cm.get_cmap('jet')
colors = ['b','g','r','c','m','y','k','w']
plt.tripcolor(mesh.nodeArray[:,0],
              mesh.nodeArray[:,1],
              mesh.elementNodesArray,
              edgecolors='k',
              linewidth=3,
              facecolors=mesh.elementMaterialTypes,
              cmap=cmap)
for si in range(mesh.nElementBoundaries_global):
    n0,n1=mesh.elementBoundaryNodesArray[si]
    k=mesh.elementBoundaryMaterialTypes[si]
    plt.plot([mesh.nodeArray[n0][0],
             mesh.nodeArray[n1][0]],
             [mesh.nodeArray[n0][1],
             mesh.nodeArray[n1][1]],
             color=colors[k],
             linewidth=3)

In [ ]:
#print("vertices="+`deditor.vertices`+
#      ", vertexFlags="+`deditor.vertexFlags`+
#      ", segments="+`deditor.segments`+
#      ", segmentFlags="+`deditor.segmentFlags`+
#      ", regions="+`deditor.regions`+
#      ", regionFlags"+`deditor.regionFlags`+
#      ", holes"+`deditor.holes`)

In [ ]:
from proteus.TransportCoefficients import TC_base
class LAD(TC_base):
    """
    The coefficients of the linear advection-diffusion equation
    """
    def __init__(self,M,A,B):
        TC_base.__init__(self, 
                         nc=1, #number of components
                         variableNames=['u'],
                         mass      = {0:{0:'linear'}},
                         advection = {0:{0:'linear'}},
                         diffusion = {0:{0:{0:'constant'}}},
                         potential = {0:{0:'u'}},
                         reaction  = {0:{0:'linear'}})
        self.M=M;
        self.A=A;
        self.B=B;
    
    def evaluate(self,t,c):
        c[('m',0)][:]         = self.M*c[('u',0)]  
        c[('dm',0,0)][:]      = self.M
        c[('f',0)][...,0]     = self.B[0]*c[('u',0)]
        c[('f',0)][...,1]     = self.B[1]*c[('u',0)]
        c[('df',0,0)][...,0]  = self.B[0]
        c[('df',0,0)][...,1]  = self.B[1]
        c[('a',0,0)][...,0,0] = self.A[0][0]
        c[('a',0,0)][...,1,1] = self.A[1][1]

In [ ]:
from proteus.iproteus import * 
Profiling.logLevel=7
Profiling.verbose=False
from proteus import default_p as p
#physics
p.name = "ladr_2d"
p.nd = 2; #Two dimensions
p.L=(1.0,1.0); 
p.T=1.0
p.domain=deditor
p.coefficients=LAD(M=1.0,
                    A=[[0.001,0.0],
                       [0.0,0.001]],
                    B=[2.0,1.0])

def getDBC(x,flag):
    if flag == 1:
        return lambda x,t: 1.0
    if flag == 2:
        return lambda x,t: 0.0

def getFlux(x,flag):
    return None
    
p.dirichletConditions = {0:getDBC}
p.advectiveFluxBoundaryConditions = {0:getFlux}
p.diffusiveFluxBoundaryConditions = {0:{0:getFlux}}
p.periodicDirichletConditions = None
class IC:
    def __init__(self):
        pass
    def uOfXT(self,x,t):
        eps=1.0e-4
        if x[0] <= 0.1+eps or x[1] <= 0.1+eps:
            return 1.0
        else:
            return 0.0

p.initialConditions  = {0:IC()}

In [ ]:
from proteus import default_n as n
import proteus as pr
n.timeIntegration = pr.TimeIntegration.BackwardEuler_cfl
n.stepController = pr.StepControl.Min_dt_cfl_controller
n.runCFL=0.9
n.femSpaces = {0:pr.FemTools.C0_AffineLinearOnSimplexWithNodalBasis}
n.elementQuadrature = pr.Quadrature.SimplexGaussQuadrature(p.nd,3)
n.elementBoundaryQuadrature = pr.Quadrature.SimplexGaussQuadrature(p.nd-1,3)
n.subgridError = pr.SubgridError.AdvectionDiffusionReaction_ASGS(p.coefficients,
                                                                 p.nd,
                                                                 lag=True)
n.shockCapturing = pr.ShockCapturing.ResGradQuad_SC(p.coefficients,
                                                    p.nd,
                                                    shockCapturingFactor=2,
                                                    lag=True)
n.numericalFluxType = pr.NumericalFlux.Advection_DiagonalUpwind_Diffusion_SIPG_exterior
n.triangleOptions="q30DenAa"+`0.001`
n.tnList=[float(i)/40.0 for i in range(11)]
n.matrix = pr.LinearAlgebraTools.SparseMatrix
n.multilevelLinearSolver = pr.LinearSolvers.LU
n.l_atol_res = 1.0e-8
n.parallelPartitioningType = pr.MeshTools.MeshParallelPartitioningTypes.node
n.nLayersOfOverlapForParallel = 1
n.periodicDirichletConditions = None

In [ ]:
from proteus import default_s,default_so, SplitOperator
so = default_so
so.name = p.name 
so.sList=[default_s]
so.tnList = n.tnList
#so.systemStepControllerType = SplitOperator.Sequential_FixedStep
#so.systemStepControllerType = SplitOperator.Sequential_FixedStep_Simple
#so.dt_system_fixed= 0.001
so.systemStepControllerType = SplitOperator.Sequential_MinModelStep
so.systemStepExact= False

In [ ]:
from proteus import NumericalSolution
ns = NumericalSolution.NS_base(so,[p],[n],so.sList,opts)

In [ ]:
failed = ns.calculateSolution('ladr_run1')
assert(not failed)

In [ ]:
%matplotlib ipympl
from matplotlib import pyplot as plt
fig = plt.figure()
x = ns.modelList[0].levelModelList[-1].mesh.nodeArray[:,0]
y = ns.modelList[0].levelModelList[-1].mesh.nodeArray[:,1]
triangles = ns.modelList[0].levelModelList[-1].mesh.elementNodesArray
u = ns.modelList[0].levelModelList[-1].u[0].dof
plt.tricontourf(x,y,triangles,u,cmap='jet')
plt.colorbar()

In [ ]:
fig = plt.figure()
plt.triplot(x,y,triangles)

In [ ]:
import matplotlib.tri as mtri
from tables import  open_file
import numpy as np
archive = open_file('ladr_2d.h5','r')
domain = p.domain
domain.x=[domain.x0, domain.y0]
domain.L=[domain.Lx, domain.Ly]
domain.nd=2
domain.getBoundingBox()
triang = mtri.Triangulation(x, y, triangles)
xg = np.linspace(0, domain.L[0], 20)
yg = np.linspace(0, domain.L[1], 20)
plt.figure()
for it,t in enumerate(n.tnList):
    u = archive.get_node("/u_t"+`it`)
    plt.clf()
    plt.xlabel(r'z[m]')
    plt.ylabel(r'x[m]')
    colors = ['b','g','r','c','m','y','k','w']
    plt.xlim(domain.x[0]-0.1*domain.L[0],domain.x[0]+domain.L[0]+0.1*domain.L[0])    
    for si,s in enumerate(domain.segments):
        plt.plot([domain.vertices[s[0]][0],
                     domain.vertices[s[1]][0]],
                    [domain.vertices[s[0]][1],
                     domain.vertices[s[1]][1]],
                    color=colors[domain.segmentFlags[si]-1],
                    linewidth=2,
                    marker='o')
    plt.tricontourf(x,y,triangles, u,levels=np.linspace(-0.2,1.2,101),cmap='jet')
    plt.colorbar()
    plt.title('T=%2.2f' % (t,))
    plt.axis('equal')
    plt.xlim((0,domain.L[0]))
    plt.savefig('phi%4.4d.png' % (it,))

In [ ]:
!rm tankPhi.mp4; ffmpeg -i phi%4d.png -vcodec libx264 tankPhi.mp4

In [ ]:
from IPython.core.display import HTML
data_uri_mp4 = open("tankPhi.mp4", "rb").read().encode("base64").replace("\n", "")
video_tag = """<video controls>
<source type ="video/mp4" src="data:video/mp4;base64,{mp4}"/>
Your browser does not support the video tag
</video>""".format(mp4=data_uri_mp4)
HTML(data=video_tag)